In [5]:
import time
import concurrent.futures
import threading
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import random

In [6]:
# 동기화를 위한 Lock 객체 생성
lock = threading.Lock()

all_problems_second = []
all_problems_third = []

def is_duplicate_problem(problems_list, link):
    return any(problem['link'] == link for problem in problems_list)

def load_problem_data(url, grade_type):
    # 각 스레드마다 독립된 웹 드라이버 인스턴스 생성
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")
    local_driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    
    try:
        local_driver.get(url)
        time.sleep(3)
        
        WebDriverWait(local_driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "td.title > div > a"))
        )
        
        html = local_driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        problems = soup.select("td.title > div > a")
        levels = soup.select("td.level")
        acceptance_rates = soup.select("td.acceptance-rate")
        
        print(f"[디버깅] URL: {url}, 가져온 문제 수: {len(problems)}")
        
        filtered_problems = []
        
        for problem, level, acceptance_rate_tag in zip(problems, levels, acceptance_rates):
            title = problem.text.strip()
            link = f"https://school.programmers.co.kr{problem['href']}"
            level_text = level.text.strip()
            acceptance_rate_text = acceptance_rate_tag.text.strip() if acceptance_rate_tag else "Unknown"

            try:
                acceptance_rate = int(''.join(filter(str.isdigit, acceptance_rate_text)))
            except ValueError:
                acceptance_rate = None

            # 동기화 처리: 리스트에 접근할 때 Lock 사용
            with lock:
                if grade_type == 'second' and not is_duplicate_problem(all_problems_second, link):
                    if acceptance_rate and 45 <= acceptance_rate <= 70 and level_text in ['Lv. 1', 'Lv. 2']: # 레벨 1, 레벨 2
                        all_problems_second.append({
                            "title": title,
                            "link": link,
                            "level": level_text,
                            "acceptance_rate": acceptance_rate_text
                        })
                elif grade_type == 'third' and not is_duplicate_problem(all_problems_third, link):
                    if acceptance_rate and 15 <= acceptance_rate <= 40 and level_text in ['Lv. 2', 'Lv. 3', 'Lv. 4']: #레벨 2, 레벨 3, 레벨 4
                        all_problems_third.append({
                            "title": title,
                            "link": link,
                            "level": level_text,
                            "acceptance_rate": acceptance_rate_text
                        })
    finally:
        local_driver.quit()  # 각 스레드의 드라이버를 종료

# 병렬 처리해서 각 페이지 동시 크롤링하도록 하는 함수
def load_problems_parallel():
    urls_second = [f"https://school.programmers.co.kr/learn/challenges?order=recent&page={i}&levels=2%2C1&languages=java%2Ccpp%2Cpython3" for i in range(1, 12)]
    urls_third = [f"https://school.programmers.co.kr/learn/challenges?order=recent&page={i}&levels=2%2C3%2C4&languages=java%2Ccpp%2Cpython3" for i in range(1, 12)]
    
    with concurrent.futures.ThreadPoolExecutor() as executor:
        # 2학년 문제
        results_second = executor.map(lambda url: load_problem_data(url, 'second'), urls_second)
        for result in results_second:
            pass

        # 3학년 문제
        results_third = executor.map(lambda url: load_problem_data(url, 'third'), urls_third)
        for result in results_third:
            pass 


In [10]:
# 문제 데이터를 병렬로 로드
# 약 1~2분 소요
load_problems_parallel()

[디버깅] URL: https://school.programmers.co.kr/learn/challenges?order=recent&page=5&levels=2%2C1&languages=java%2Ccpp%2Cpython3, 가져온 문제 수: 20
[디버깅] URL: https://school.programmers.co.kr/learn/challenges?order=recent&page=11&levels=2%2C1&languages=java%2Ccpp%2Cpython3, 가져온 문제 수: 3
[디버깅] URL: https://school.programmers.co.kr/learn/challenges?order=recent&page=1&levels=2%2C1&languages=java%2Ccpp%2Cpython3, 가져온 문제 수: 20
[디버깅] URL: https://school.programmers.co.kr/learn/challenges?order=recent&page=7&levels=2%2C1&languages=java%2Ccpp%2Cpython3, 가져온 문제 수: 20
[디버깅] URL: https://school.programmers.co.kr/learn/challenges?order=recent&page=8&levels=2%2C1&languages=java%2Ccpp%2Cpython3, 가져온 문제 수: 20
[디버깅] URL: https://school.programmers.co.kr/learn/challenges?order=recent&page=6&levels=2%2C1&languages=java%2Ccpp%2Cpython3, 가져온 문제 수: 20
[디버깅] URL: https://school.programmers.co.kr/learn/challenges?order=recent&page=10&levels=2%2C1&languages=java%2Ccpp%2Cpython3, 가져온 문제 수: 20
[디버깅] URL: https://school.

In [11]:
print("2학년 문제 개수: "+str(len(all_problems_second)))
print("3학년 문제 개수: "+str(len(all_problems_third)))

2학년 문제 개수: 106
3학년 문제 개수: 80


In [12]:
if len(all_problems_second) >= 2:
    random_problems = random.sample(all_problems_second, 2)  # 중복 없이 두 문제 선택
    print("2학년 문제")
    print(f"문제 제목 1: {random_problems[0]['title']}")
    print(f"문제 링크: {random_problems[0]['link']}")
    print(f"난이도: {random_problems[0]['level']}")
    print(f"정답률: {random_problems[0]['acceptance_rate']}\n")
    
    print(f"문제 제목 2: {random_problems[1]['title']}")
    print(f"문제 링크: {random_problems[1]['link']}")
    print(f"난이도: {random_problems[1]['level']}")
    print(f"정답률: {random_problems[1]['acceptance_rate']}\n")
    print("**************************************************\n")
elif len(all_problems_second) == 1:
    print("2학년 문제")
    print(f"문제 제목: {all_problems_second[0]['title']}")
    print(f"문제 링크: {all_problems_second[0]['link']}")
    print(f"난이도: {all_problems_second[0]['level']}")
    print(f"정답률: {all_problems_second[0]['acceptance_rate']}\n")
    print("**************************************************\n")
else:
    print("2학년 문제를 찾을 수 없습니다.")


if all_problems_third:
    random_problem = random.choice(all_problems_third)
    print("3학년 문제")
    print(f"문제 제목: {random_problem['title']}")
    print(f"문제 링크: {random_problem['link']}")
    print(f"난이도: {random_problem['level']}")
    print(f"정답률: {random_problem['acceptance_rate']}")
else:
    print("3학년 문제를 찾을 수 없습니다.")


2학년 문제
문제 제목 1: 가장 큰 정사각형 찾기
문제 링크: https://school.programmers.co.kr/learn/courses/30/lessons/12905
난이도: Lv. 2
정답률: 46%

문제 제목 2: 키패드 누르기
문제 링크: https://school.programmers.co.kr/learn/courses/30/lessons/67256
난이도: Lv. 1
정답률: 52%

**************************************************

3학년 문제
문제 제목: 징검다리
문제 링크: https://school.programmers.co.kr/learn/courses/30/lessons/43236
난이도: Lv. 4
정답률: 27%
